In [ ]:
#Don't forget to save your work to your local computer and push to GitHub.
#File -> Download as -> Notebook
#Move file from download folder to local repo clone
#git add filename.ipynb
#git commit -m "put a specific and informative message here of what you worked on"
#git push

In [114]:
import pandas
import os

# This query represents dataset "Concept Set Exploration" for domain "condition" and was generated for All of Us Controlled Tier Dataset v7
dataset_95902282_condition_sql = """
    SELECT
        c_occurrence.person_id,
        c_standard_concept.concept_name as standard_concept_name,
        c_occurrence.condition_start_datetime 
    FROM
        ( SELECT
            * 
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.condition_occurrence` c_occurrence 
        WHERE
            (
                condition_concept_id IN  (
                    SELECT
                        DISTINCT c.concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                    JOIN
                        (
                            select
                                cast(cr.id as string) as id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                            WHERE
                                concept_id IN (
                                    36712702, 4014295, 432441, 44784550, 45757175, 45757176
                                ) 
                                AND full_text LIKE '%_rank1]%'
                        ) a 
                            ON (
                                c.path LIKE CONCAT('%.',
                            a.id,
                            '.%') 
                            OR c.path LIKE CONCAT('%.',
                            a.id) 
                            OR c.path LIKE CONCAT(a.id,
                            '.%') 
                            OR c.path = a.id) 
                        WHERE
                            is_standard = 1 
                            AND is_selectable = 1
                        )
                )  
                AND (
                    c_occurrence.PERSON_ID IN (
                        SELECT
                            distinct person_id  
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                        WHERE
                            cb_search_person.person_id IN (
                                SELECT
                                    criteria.person_id 
                                FROM
                                    (SELECT
                                        DISTINCT person_id,
                                        entry_date,
                                        concept_id 
                                    FROM
                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                    WHERE
                                        (
                                            concept_id IN (
                                                SELECT
                                                    DISTINCT c.concept_id 
                                                FROM
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                JOIN
                                                    (
                                                        select
                                                            cast(cr.id as string) as id 
                                                        FROM
                                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                        WHERE
                                                            concept_id IN (36712702, 45757176, 4014295) 
                                                            AND full_text LIKE '%_rank1]%'
                                                    ) a 
                                                        ON (
                                                            c.path LIKE CONCAT('%.',
                                                        a.id,
                                                        '.%') 
                                                        OR c.path LIKE CONCAT('%.',
                                                        a.id) 
                                                        OR c.path LIKE CONCAT(a.id,
                                                        '.%') 
                                                        OR c.path = a.id) 
                                                    WHERE
                                                        is_standard = 1 
                                                        AND is_selectable = 1
                                                    ) 
                                                    AND is_standard = 1 
                                            )
                                        ) criteria 
                                    ) ))
                        ) c_occurrence 
                    LEFT JOIN
                        `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_standard_concept 
                            ON c_occurrence.condition_concept_id = c_standard_concept.concept_id"""

dataset_95902282_condition_df = pandas.read_gbq(
    dataset_95902282_condition_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_95902282_condition_df.sample(10)

Downloading:   0%|          | 0/390805 [00:00<?, ?rows/s]

,person_id,standard_concept_name,condition_start_datetime
357809,3357733,Third trimester pregnancy,2017-06-06 00:00:00+00:00
280724,5250713,Third trimester pregnancy,2016-11-22 23:18:00+00:00
228381,1127634,Single live birth,2020-04-17 10:28:00+00:00
69926,1818320,"Gestation period, 34 weeks",2019-01-14 19:13:00+00:00
273200,2068119,Single live birth,2010-07-08 06:00:00+00:00
78956,2340829,"Gestation period, 39 weeks",2019-06-28 05:00:00+00:00
178803,1016283,Second trimester pregnancy,2019-08-29 00:00:00+00:00
52938,2596557,"Gestation period, 12 weeks",2018-01-24 05:00:00+00:00
386637,1213039,Third trimester pregnancy,2021-09-07 13:39:00+00:00
165077,1765735,Second trimester pregnancy,2017-09-06 00:00:00+00:00


In [115]:
#Rename dataset
long_data = dataset_95902282_condition_df

long_data.info

<bound method DataFrame.info of         person_id         standard_concept_name  condition_start_datetime
0         5433458  Gestation less than 20 weeks 2022-01-06 00:00:00+00:00
1         5433458  Gestation less than 20 weeks 2021-12-09 00:00:00+00:00
2         5433458  Gestation less than 20 weeks 2021-11-12 00:00:00+00:00
3         5433458  Gestation less than 20 weeks 2021-12-09 00:00:00+00:00
4         5433458  Gestation less than 20 weeks 2021-11-04 00:00:00+00:00
...           ...                           ...                       ...
390800    2814607     Third trimester pregnancy 2022-05-12 14:56:00+00:00
390801    1777594     Third trimester pregnancy 2019-10-24 15:00:00+00:00
390802    4382057     Gestation period, 8 weeks 2021-09-23 09:29:00+00:00
390803    1024748     Third trimester pregnancy 2022-05-27 15:20:00+00:00
390804    1394791     Third trimester pregnancy 2020-08-06 09:00:00+00:00

[390805 rows x 3 columns]>

In [116]:
#Review entries in standard_concept_name and remove irrelevant entries

unique_values = long_data['standard_concept_name'].unique()

#print("List of all possible values for 'standard_concept_name':")
#for value in unique_values:
#    print(value)

values_to_drop = ['Gestation less than 20 weeks', 'Gestation less than 9 weeks']
values_to_drop += ['Finding of length of gestation', 'Gestation less than 24 weeks']
values_to_drop += ['First trimester pregnancy', 'Second trimester pregnancy', 'Third trimester pregnancy']

# Use the 'drop' method to remove rows with specific values in 'standard_concept_name'
drop_long_data = long_data[~long_data['standard_concept_name'].isin(values_to_drop)]

print("DataFrame after dropping rows with specific values in 'standard_concept_name':")
print(drop_long_data)

DataFrame after dropping rows with specific values in 'standard_concept_name':
        person_id       standard_concept_name  condition_start_datetime
5         2003968  Gestation period, 35 weeks 2021-02-05 09:10:13+00:00
6         2125781  Gestation period, 35 weeks 2022-05-25 08:59:09+00:00
7         2312731  Gestation period, 35 weeks 2019-02-07 08:54:19+00:00
8         2641486  Gestation period, 35 weeks 2018-08-14 03:38:01+00:00
9         6665006  Gestation period, 35 weeks 2022-03-03 23:29:51+00:00
...           ...                         ...                       ...
390746    2393893   Gestation period, 8 weeks 2017-05-09 19:53:00+00:00
390753    1056354   Gestation period, 8 weeks 2019-12-24 15:02:00+00:00
390764    2950768   Gestation period, 8 weeks 2018-11-19 05:00:00+00:00
390799    3279787   Gestation period, 8 weeks 2021-01-20 08:47:00+00:00
390802    4382057   Gestation period, 8 weeks 2021-09-23 09:29:00+00:00

[182208 rows x 3 columns]


In [117]:
#Convert condition_start_datetime cells to contain only date information and not time, as time is irrelevant to our question
#Make a new copy of the dataset and drop the old datetime column, retaining new date column

# Convert the datetime column to a datetime type
drop_long_data.loc[:,'condition_start_datetime'] = pd.to_datetime(drop_long_data['condition_start_datetime'])

#Create a new date only column
drop_long_data.loc[:,'condition_start_date'] = drop_long_data['condition_start_datetime'].dt.date

# Drop the datetime_column
date_only_long_data = drop_long_data.drop(columns=['condition_start_datetime']).copy()

# Display the new DataFrame with all columns except datetime_column
print("New DataFrame with All Columns Except datetime_column:")
print(date_only_long_data)



New DataFrame with All Columns Except datetime_column:
        person_id       standard_concept_name condition_start_date
5         2003968  Gestation period, 35 weeks           2021-02-05
6         2125781  Gestation period, 35 weeks           2022-05-25
7         2312731  Gestation period, 35 weeks           2019-02-07
8         2641486  Gestation period, 35 weeks           2018-08-14
9         6665006  Gestation period, 35 weeks           2022-03-03
...           ...                         ...                  ...
390746    2393893   Gestation period, 8 weeks           2017-05-09
390753    1056354   Gestation period, 8 weeks           2019-12-24
390764    2950768   Gestation period, 8 weeks           2018-11-19
390799    3279787   Gestation period, 8 weeks           2021-01-20
390802    4382057   Gestation period, 8 weeks           2021-09-23

[182208 rows x 3 columns]


/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [118]:
#Remove duplicates that have the same person_id, standard_concept_name, and condition_start_date
#(Note that this will retain duplicates of same person_id and standard_concept_name but different condition_start_date)

#Drop rows with the same 'person_id', 'standard_concept_name', and 'condition_start_date', keeping first instance
date_only_long_data_no_triple_duplicates = date_only_long_data.drop_duplicates(subset=['person_id', 'standard_concept_name', 'condition_start_date'], keep='first')

print("Dataframe after dropping duplicate rows:")
print(date_only_long_data_no_triple_duplicates)

#Double check for duplicates
#check_duplicates = filtered_long_data_no_duplicates.duplicated(subset=['person_id', 'standard_concept_name', 'condition_start_date'], keep='first')
#Print any duplicate rows
#duplicate_rows_check = filtered_long_data_no_duplicates[check_duplicates]
#print("Duplicate Rows in the Resulting DataFrame:")
#print(duplicate_rows_check)




Dataframe after dropping duplicate rows:
        person_id       standard_concept_name condition_start_date
5         2003968  Gestation period, 35 weeks           2021-02-05
6         2125781  Gestation period, 35 weeks           2022-05-25
7         2312731  Gestation period, 35 weeks           2019-02-07
8         2641486  Gestation period, 35 weeks           2018-08-14
9         6665006  Gestation period, 35 weeks           2022-03-03
...           ...                         ...                  ...
390735    1350249   Gestation period, 8 weeks           2019-10-18
390743    3392412   Gestation period, 8 weeks           2017-09-29
390753    1056354   Gestation period, 8 weeks           2019-12-24
390764    2950768   Gestation period, 8 weeks           2018-11-19
390799    3279787   Gestation period, 8 weeks           2021-01-20

[143195 rows x 3 columns]


In [119]:
#Explore duplicate entries that differ in condition_start_datetime
duplicates = date_only_long_data_no_triple_duplicates.duplicated(subset=['person_id', 'standard_concept_name'], keep=False)
different_date_duplicates = date_only_long_data_no_triple_duplicates[duplicates]

print("Number of Rows that are Duplicates in 'person_id' and 'standard_concept_name' but have Different 'condition_start_date':", len(different_date_duplicates))

# Identify and count duplicate rows based on 'person_id' and 'standard_concept_name' while considering differences in 'condition_start_date'
duplicates = date_only_long_data_no_triple_duplicates.duplicated(subset=['person_id', 'standard_concept_name'], keep=False)
different_date_duplicates = date_only_long_data_no_triple_duplicates[duplicates]

# Create a DataFrame of the original rows along with their duplicate counterparts
matching_pairs = date_only_long_data_no_triple_duplicates[duplicates | duplicates.shift(-1)]

# Sort the DataFrame by 'person_id'
matching_pairs = matching_pairs.sort_values(by='person_id')

print("DataFrame of Duplicate Rows and Their Matching Pairs (Sorted by 'person_id'):")
print(matching_pairs)


#See what is the max number of duplicates for a given person_id and standard_concept_name combination
# Identify and count duplicate rows based on 'person_id' and 'standard_concept_name'
duplicates = date_only_long_data_no_triple_duplicates.duplicated(subset=['person_id', 'standard_concept_name'], keep=False)

# Create a DataFrame of the duplicate rows
duplicate_rows = date_only_long_data_no_triple_duplicates[duplicates]

# Group by 'person_id' and 'standard_concept_name' and count the duplicates
duplicate_count = duplicate_rows.groupby(['person_id', 'standard_concept_name']).size()

# Find the combination with the most duplicates
max_duplicates = duplicate_count.idxmax()
max_count = duplicate_count.max()

print("Combination with the Most Duplicates:")
print("person_id:", max_duplicates[0])
print("standard_concept_name:", max_duplicates[1])
print("Number of Duplicates:", max_count)



Number of Rows that are Duplicates in 'person_id' and 'standard_concept_name' but have Different 'condition_start_date': 82422
DataFrame of Duplicate Rows and Their Matching Pairs (Sorted by 'person_id'):
        person_id       standard_concept_name condition_start_date
239984    1000104           Single live birth           2005-06-20
259407    1000104           Single live birth           2005-06-23
254587    1000109           Single live birth           2001-11-13
232688    1000109           Single live birth           2000-03-13
260396    1000109           Single live birth           2001-11-16
...           ...                         ...                  ...
260627    9989704           Single live birth           2007-11-02
269688    9998092           Single live birth           1994-12-17
80737     9998770  Gestation period, 39 weeks           2016-10-25
247054    9998770           Single live birth           2016-10-25
257228    9998770           Single live birth           20

In [120]:
#Pivot from long to wide format once ALL DUPLICATES HAVE BEEN RESOLVED

#wide_data = long_data.pivot(index='person_id', columns='standard_concept_name', values='condition_start_datetime')

#wide_data.sample(20)


In [121]:
#view all entries for a specific person_id to explore duplicates, etc


#person_df = df[df['person_id'] == 5433458]
#person_df


#person_df = date_only_long_data_no_triple_duplicates[date_only_long_data_no_triple_duplicates['person_id'] == 1302966]
#person_df


person_df = date_only_long_data_no_triple_duplicates[date_only_long_data_no_triple_duplicates['person_id'] == 1934256	]
person_df = person_df.sort_values(by='condition_start_date')
person_df



,person_id,standard_concept_name,condition_start_date
257993,1934256,Single live birth,2013-12-17
242563,1934256,Single live birth,2013-12-18
269358,1934256,Single live birth,2013-12-23
58135,1934256,Preterm labor with preterm delivery,2015-12-10
58104,1934256,Preterm labor with preterm delivery,2015-12-23
58087,1934256,Preterm labor with preterm delivery,2015-12-24
58141,1934256,Preterm labor with preterm delivery,2016-01-06
58086,1934256,Preterm labor with preterm delivery,2016-01-22
22692,1934256,"Gestation period, 21 weeks",2016-01-23
25007,1934256,"Gestation period, 21 weeks",2016-01-26


In [122]:
#view all entries for standard_concept_name

import pandas as pd

#Create a dictionary to store concept names
concept_dict = {}

#Iterate through column 'standard_concept_name' 
for concept in date_only_long_data_no_triple_duplicates['standard_concept_name']:
    if concept in concept_dict:
        concept_dict[concept] += 1
    else:
        concept_dict[concept] = 1

#print nicely
for key, value in concept_dict.items():
    print(f'{key}: {value}')


Gestation period, 35 weeks: 4700
Gestation period, 18 weeks: 2703
Gestation period, 13 weeks: 1862
Term pregnancy: 3
Gestation period, 36 weeks: 6376
Gestation period, 23 weeks: 1692
Gestation period, 21 weeks: 2105
Gestation period, 38 weeks: 7400
Gestation period, 37 weeks: 7332
Gestation period, 25 weeks: 1566
Gestation period, 14 weeks: 1157
Gestation period, 42 weeks: 15
Preterm spontaneous labor with preterm delivery: 425
Gestation period, 12 weeks: 3479
Gestation period, 22 weeks: 2023
Preterm labor in second trimester with preterm delivery in second trimester: 150
Preterm labor with preterm delivery: 68
Gestation period, 16 weeks: 1508
Gestation period, 31 weeks: 2745
Gestation period, 30 weeks: 2600
Gestation period, 19 weeks: 2613
Gestation period, 34 weeks: 4411
Gestation period, 39 weeks: 9860
Gestation period, 15 weeks: 1104
Gestation period, 9 weeks: 34
Gestation period, 10 weeks: 1552
Gestation period, 17 weeks: 1339
Gestation period, 33 weeks: 3714
Gestation period, 29 

In [123]:
#Create code to calculate conception date from standard_concept_name that gives gestational age

from datetime import timedelta

# Define a function to calculate the conception date
def calculate_conception(row):
    # Split the 'standard_concept_name' to extract the number of weeks
    concept_name_parts = row['standard_concept_name'].split()
    if len(concept_name_parts) == 4 and concept_name_parts[0] == "Gestation":
        try:
            weeks = int(concept_name_parts[2])
            # Calculate the conception date by subtracting 'weeks' from 'condition_start_datetime'
            conception_date = row['condition_start_date'] - timedelta(weeks=weeks)
            return conception_date
        except ValueError:
            pass
    return None

# Apply the function to create the 'conception_date' column
date_only_long_data_no_triple_duplicates['conception_date'] = date_only_long_data_no_triple_duplicates.apply(calculate_conception, axis=1)

date_only_long_data_no_triple_duplicates.sample(10)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,person_id,standard_concept_name,condition_start_date,conception_date
105139,1150781,"Gestation period, 29 weeks",2018-11-19,2018-04-30
259213,6556799,Single live birth,2001-08-24,None
355419,2958170,"Gestation period, 8 weeks",2020-10-13,2020-08-18
23391,2784938,"Gestation period, 21 weeks",2017-02-17,2016-09-23
22834,3101207,"Gestation period, 23 weeks",2019-03-29,2018-10-19
68763,3012054,"Gestation period, 19 weeks",2016-09-20,2016-05-10
17135,1798145,"Gestation period, 36 weeks",2017-07-17,2016-11-07
87556,1305876,"Gestation period, 39 weeks",2016-11-14,2016-02-15
90802,1412094,"Gestation period, 10 weeks",2016-06-09,2016-03-31
239179,3458725,Single live birth,2008-01-01,None


In [124]:
#If we revisit our person from above, we will see that we calculated a conception date, but there is some variance.
#Some entries seem wrong (9/9 conception when all the others are august, but others range from 8/6 - 8/18)

person_df = filtered_long_data_no_duplicates[filtered_long_data_no_duplicates['person_id'] == 5433458]
person_df

person_df.dropna(subset=['conception_date'])

,person_id,standard_concept_name,condition_start_date,conception_date
7361,5433458,"Gestation period, 13 weeks",2021-12-09,2021-09-09
12139,5433458,"Gestation period, 13 weeks",2021-11-12,2021-08-13
13907,5433458,"Gestation period, 36 weeks",2022-04-27,2021-08-18
19783,5433458,"Gestation period, 36 weeks",2022-04-22,2021-08-13
24117,5433458,"Gestation period, 21 weeks",2022-01-06,2021-08-12
29836,5433458,"Gestation period, 38 weeks",2022-05-05,2021-08-12
31302,5433458,"Gestation period, 38 weeks",2022-05-09,2021-08-16
53741,5433458,"Gestation period, 12 weeks",2021-11-04,2021-08-12
62933,5433458,"Gestation period, 30 weeks",2022-03-09,2021-08-11
72081,5433458,"Gestation period, 34 weeks",2022-04-07,2021-08-12


In [11]:
#The earlier the ultrasound, the more accurate (before 14 weeks, and especially at 8 weeks). American College of Obstetricians and Gynecologists. 
#8 weeks is a common time in US healthcare to get US and measurements, and we have n = 7k.
#If we can't calculate single live birth from the 8 week entry, then let's choose 10, 12, 13, weeks etc and prioritize
#which entries are used to calculate.

#I'm not yet sure how to handle instances like the person above who the first two lines are both week 13, but at appointments 
#4 weeks apart; one is clearly wrong. fortunately in this case we would use her week 8 data anyway....

In [148]:
df = date_only_long_data_no_triple_duplicates.copy()
df.sample(10)

,person_id,standard_concept_name,condition_start_date,conception_date
244812,2002083,Single live birth,2012-12-21,None
75337,4565143,"Gestation period, 34 weeks",2020-06-19,2019-10-25
63139,1335231,"Gestation period, 31 weeks",2021-12-14,2021-05-11
287261,8648990,"Gestation period, 8 weeks",2019-03-18,2019-01-21
103679,2891311,"Gestation period, 29 weeks",2016-10-11,2016-03-22
69861,1291532,"Gestation period, 19 weeks",2019-04-05,2018-11-23
30587,1995850,"Gestation period, 38 weeks",2019-09-30,2019-01-07
19744,1904122,"Gestation period, 36 weeks",2016-10-16,2016-02-07
260703,1312605,Single live birth,2016-11-19,None
81689,1270278,"Gestation period, 39 weeks",2018-01-24,2017-04-26


In [195]:
#Okay it is working maybe...
df = date_only_long_data_no_triple_duplicates.copy()
df = df.sort_values(by=['person_id', 'condition_start_date'])

# List of gestational periods in priority order
gestational_periods = [
    'Gestation period, 8 weeks',
    'Gestation period, 9 weeks',
    'Gestation period, 10 weeks',
    'Gestation period, 11 weeks',
    'Gestation period, 12 weeks',
    'Gestation period, 13 weeks',
    'Gestation period, 42 weeks',
    'Gestation period, 41 weeks',
    'Gestation period, 40 weeks',
    'Gestation period, 39 weeks',
    'Gestation period, 38 weeks',
    'Gestation period, 37 weeks',
]

# Initialize variables to store the most recent "Gestation period" and birth dates
most_accurate_conception_date = None
birth_dates = []

# Iterate through the DataFrame
current_person_id = None  # Variable to track the current person_id
for idx, row in df.iterrows():
    if row['person_id'] != current_person_id:
        most_accurate_conception_date = None  # Reset for a new person
        current_person_id = row['person_id']  # Update current person_id
    
    for period in gestational_periods:
        if row['standard_concept_name'] == period:
            if most_accurate_conception_date is None:
                most_accurate_conception_date = row['conception_date']
    
    if row['standard_concept_name'] == 'Single live birth':
        birth_dates.append((idx, row['condition_start_date']))
        
        if most_accurate_conception_date is not None and pd.isna(df.at[idx, 'gestational_age_at_birth']):
            last_idx, last_birth_date = birth_dates[-1]
            gestational_age = (pd.to_datetime(last_birth_date) - pd.to_datetime(most_accurate_conception_date)).days / 7
            df.at[last_idx, 'gestational_age_at_birth'] = gestational_age
            most_accurate_conception_date = None
        else:
            last_idx, _ = birth_dates[-1]
            df.at[last_idx, 'gestational_age_at_birth'] = np.nan

# Display the updated DataFrame
df.sample(20)



,person_id,standard_concept_name,condition_start_date,conception_date,gestational_age_at_birth
236658,1582522,Single live birth,2005-05-12,None,NaN
115128,3378643,"Gestation period, 40 weeks",2022-06-14,2021-09-07,NaN
82523,1540446,"Gestation period, 39 weeks",2019-01-22,2018-04-24,NaN
45496,1807339,"Gestation period, 37 weeks",2018-05-17,2017-08-31,NaN
253145,1328237,Single live birth,2013-12-19,None,NaN
266962,1738878,Single live birth,2020-02-11,None,39.0
35074,1227604,"Gestation period, 38 weeks",2020-01-17,2019-04-26,NaN
108204,8543938,"Gestation period, 27 weeks",2021-04-16,2020-10-09,NaN
246739,1435169,Single live birth,2014-02-15,None,NaN
6622,2688088,"Gestation period, 18 weeks",2019-06-06,2019-01-31,NaN


In [190]:
added_gestational_ages = df['gestational_age_at_birth'].count()
print("Number of gestational ages added:", added_gestational_ages)

Number of gestational ages added: 19394


In [199]:
#Checking numbers to see how the loop is working....

#Missing gestation_age_at_birth (for example, birth occurred but no previous records attached)
missing_gestational_age = len(df[df['standard_concept_name'] == 'Single live birth'][df['gestational_age_at_birth'].isna()])
print("Number of 'Single live birth' entries with missing gestational_age_at_birth:", missing_gestational_age)

# Filter and delete entries with negative or greater than 45 weeks gestational age
negative_gestational_age = len(df[df['gestational_age_at_birth'] < 0])
greater_than_45_weeks = len(df[df['gestational_age_at_birth'] > 45])
filtered_df = df[(df['gestational_age_at_birth'] >= 0) & (df['gestational_age_at_birth'] <= 45)]

# Filter the DataFrame for 'Single live birth' with gestational age less than 37 weeks and full term
single_live_births_less_than_37 = len(filtered_df[(filtered_df['standard_concept_name'] == 'Single live birth') & (filtered_df['gestational_age_at_birth'] < 37)])
single_live_births_full_term = len(filtered_df[(filtered_df['standard_concept_name'] == 'Single live birth') & (filtered_df['gestational_age_at_birth'] >= 37)])



# Display the counts of entries deleted
print("Number of entries with negative gestational age deleted:", negative_gestational_age)
print("Number of entries with gestational age greater than 45 weeks deleted:", greater_than_45_weeks)
print("Number of single live births that are full term >= 37 weeks:", single_live_births_full_term)
print("Number of single live births that are preterm < 37 weeks:", single_live_births_less_than_37)
print("Percentage of dataset that is preterm:", single_live_births_less_than_37 / single_live_births_full_term * 100)

Number of 'Single live birth' entries with missing gestational_age_at_birth: 16683
Number of entries with negative gestational age deleted: 0
Number of entries with gestational age greater than 45 weeks deleted: 1170
Number of single live births that are full term >= 37 weeks: 16634
Number of single live births that are preterm < 37 weeks: 998
Percentage of dataset that is preterm: 5.999759528676205


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


In [200]:
# Filter the DataFrame to select rows with "Single live birth" and terms to explore (e.g. why neg ages, why over-term etc)
explore_gestational_age_df = df[(df['standard_concept_name'] == 'Single live birth') & (df['gestational_age_at_birth'] > 45)]

# Get the unique person_id values for those rows
person_ids_with_expl_values = explore_gestational_age_df['person_id'].unique()

# Display the list of person_id values
print("person_id values with explore gestational ages in 'Single live birth' entries:")
print(person_ids_with_expl_values)


person_id values with explore gestational ages in 'Single live birth' entries:
<IntegerArray>
[1000195, 1009088, 1010363, 1010808, 1011255, 1012047, 1013211, 1015897,
 1016951, 1017057,
 ...
 9707760, 9756046, 9791266, 9797895, 9804305, 9817644, 9849830, 9915081,
 9923440, 9930591]
Length: 1147, dtype: Int64


In [201]:
#Why does this person have very long term
person_df = df[df['person_id'] == 1000195]
person_df

,person_id,standard_concept_name,condition_start_date,conception_date,gestational_age_at_birth,birth_class
233571,1000195,Single live birth,2013-05-23,None,NaN,Unknown
234512,1000195,Single live birth,2013-05-24,None,NaN,Unknown
272428,1000195,Single live birth,2013-05-25,None,NaN,Unknown
237194,1000195,Single live birth,2015-06-27,None,NaN,Unknown
243813,1000195,Single live birth,2015-06-29,None,NaN,Unknown
7809,1000195,"Gestation period, 13 weeks",2020-12-28,2020-09-28,NaN,Unknown
6379,1000195,"Gestation period, 13 weeks",2021-01-28,2020-10-29,NaN,Unknown
11179,1000195,"Gestation period, 18 weeks",2021-03-04,2020-10-29,NaN,Unknown
112941,1000195,"Gestation period, 20 weeks",2021-03-19,2020-10-30,NaN,Unknown
55910,1000195,"Gestation period, 22 weeks",2021-04-01,2020-10-29,NaN,Unknown


In [ ]:
#The above numbers for preterm births are those listed as single live birth that we then calculated preterm age based
#on gestational findings earlier in records. Now we can convert term and preterm into categories in another column, and
#add preterm birth findings and see if that increases our percentage of dataset that is preterm.

In [197]:
# Create a new column "birth_class" and set it to 'Unknown' initially
df['birth_class'] = 'Unknown'

# Update "birth_class" for rows with "Single live birth" based on gestational age
df.loc[(df['standard_concept_name'] == 'Single live birth') & (df['gestational_age_at_birth'] >= 37), 'birth_class'] = 'Term'
df.loc[(df['standard_concept_name'] == 'Single live birth') & (df['gestational_age_at_birth'] < 37), 'birth_class'] = 'Preterm'


# Display the updated DataFrame
df.sample(20)


,person_id,standard_concept_name,condition_start_date,conception_date,gestational_age_at_birth,birth_class
104780,2251397,"Gestation period, 29 weeks",2019-10-15,2019-03-26,NaN,Unknown
58016,2002194,Preterm labor in second trimester with preterm...,2019-07-05,None,NaN,Unknown
113707,1437457,"Gestation period, 40 weeks",2019-02-04,2018-04-30,NaN,Unknown
60009,2121601,"Gestation period, 16 weeks",2016-07-26,2016-04-05,NaN,Unknown
59587,3043424,"Gestation period, 16 weeks",2018-09-18,2018-05-29,NaN,Unknown
105194,2508163,"Gestation period, 29 weeks",2021-03-26,2020-09-04,NaN,Unknown
253676,1954487,Single live birth,2017-10-14,None,38.571429,Term
62444,1488159,"Gestation period, 31 weeks",2017-05-09,2016-10-04,NaN,Unknown
81648,2699226,"Gestation period, 39 weeks",2019-07-29,2018-10-29,NaN,Unknown
268625,1837867,Single live birth,2015-07-06,None,NaN,Unknown


,person_id,standard_concept_name,condition_start_date,conception_date,gestational_age_at_birth,birth_class
255221,2182567,Single live birth,2007-08-26,None,NaN,Unknown
239574,2182567,Single live birth,2009-06-09,None,NaN,Unknown
255180,2182567,Single live birth,2011-03-17,None,NaN,Unknown
269122,2182567,Single live birth,2014-06-06,None,NaN,Unknown
79479,2182567,"Gestation period, 39 weeks",2016-11-30,2016-03-02,NaN,Unknown
244169,2182567,Single live birth,2016-11-30,None,39.0,Term
